In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Sustainable.csv')

In [3]:
df = df.replace("'NULL'",None)
df

,Country Name,Country Code,Time,Time Code,Access to Clean Fuels and Technologies for cooking (% of total population) [2.1_ACCESS.CFT.TOT],Access to electricity (% of rural population with access) [1.2_ACCESS.ELECTRICITY.RURAL],Access to electricity (% of total population) [1.1_ACCESS.ELECTRICITY.TOT],Access to electricity (% of urban population with access) [1.3_ACCESS.ELECTRICITY.URBAN],Energy intensity level of primary energy (MJ/2011 USD PPP) [6.1_PRIMARY.ENERGY.INTENSITY],Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT],Renewable electricity share of total electricity output (%) [4.1_SHARE.RE.IN.ELECTRICITY],Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION],Renewable energy share of TFEC (%) [2.1_SHARE.TOTAL.RE.IN.TFEC],Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT],Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]
0,Afghanistan,AFG,1996,YR1996,None,None,0.01,62.65266037,1.548476916,475,70.37037037,9304.887,37.9457475,675,24521.5541
1,Afghanistan,AFG,1997,YR1997,None,None,0.01,64.40988922,1.599236643,485,72.3880597,9861.213,41.43260087,670,23800.61303
2,Afghanistan,AFG,1998,YR1998,None,None,0.021977138,66.16093445,1.648977886,495,74.43609023,10254.29,44.09433685,665,23255.34464
3,Afghanistan,AFG,1999,YR1999,None,None,0.179634586,67.90476227,1.570681102,505,73.72262774,10765.99,52.18577437,685,20630.12407
4,Afghanistan,AFG,2000,YR2000,8.8,None,0.959755898,69.64098358,1.686173124,515.7,74.98909408,11301.39,54.24312637,687.7,20834.69512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5434,Zimbabwe,ZWE,2012,YR2012,30.02,12.98164418,36.72887802,85.30677032,18.43724649,5558,60.39334999,304744.9,78.01970206,9203,390599.9279
5435,Zimbabwe,ZWE,2013,YR2013,29.88,13.66991771,37.07681274,85.35147858,18.15296658,5162,53.93939394,311169.4,79.56109383,9570,391107.4937
5436,Zimbabwe,ZWE,2014,YR2014,29.63,9.8,32.3,83.4,15.74958139,5575,55.60542589,320293.3,81.0492915,10026,395183.3435
5437,Zimbabwe,ZWE,2015,YR2015,29.36,9.7,33.7,81.2,15.80389705,5119,52.72427644,324422.6,81.79780855,9709,396615.2709


In [4]:
countries = pd.DataFrame({
    'Country Name': df['Country Name'].unique(),
    'Country Code': df['Country Code'].unique()
    })

country_names = countries['Country Name'].values.tolist()
country_codes = countries['Country Code'].values.tolist()

In [5]:
years = pd.DataFrame({
    'year': df['Time'].unique(),
    'code': df['Time Code'].unique()
    })

years_values = years['year'].values.tolist()
years_codes_values = years['code'].values.tolist()

In [8]:
import psycopg2

def connect_to_db():
    return psycopg2.connect(host="localhost", database="sustainability", user="stanleymyersomulloy")

In [10]:
def add_countries(country_names, country_codes):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()
        length = len(country_names)
        print('connected')
        for i in range(length):
            cur.execute("INSERT INTO country (name, code) VALUES (%s, %s);", [country_names[i], country_codes[i]])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
add_countries(country_names, country_codes)

connected


In [11]:
def add_years(years_values, years_codes_values):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()
        print('connected')
        for i in range(len(years_values)):
            cur.execute("INSERT INTO years (year, code) VALUES (%s, %s);", [years_values[i],years_codes_values[i]])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

add_years(years_values, years_codes_values)

connected


In [14]:
def populate_country_year(df):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()

        print('connected')
        for i in range(len(df)):
            row_dict = df.loc[i]
            country_code = row_dict['Country Code']
            year_code = row_dict['Time Code']
    
            cur.execute("INSERT INTO country_year (year_code, country_code) VALUES (%s, %s);", [year_code,country_code])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

populate_country_year(df)

connected


In [24]:
def populate_electricity_access(df):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()

        print('connected')
        for i in range(len(df)):

            row_dict = df.loc[i]
            country_code = row_dict['Country Code']
            year_code = row_dict['Time Code']
            electricity_urban = row_dict['Access to electricity (% of urban population with access) [1.3_ACCESS.ELECTRICITY.URBAN]']
            electricity_rural = row_dict['Access to electricity (% of rural population with access) [1.2_ACCESS.ELECTRICITY.RURAL]']
            electricity_total = row_dict['Access to electricity (% of total population) [1.1_ACCESS.ELECTRICITY.TOT]']

            if electricity_urban or electricity_rural or electricity_total:
                cur.execute(
                    "INSERT INTO electricity_access \
                        (country_year_id, electricity_urban, electricity_rural, electricity_total) \
                    VALUES ( \
                        (SELECT country_year.id FROM country_year WHERE country_code = %s AND year_code = %s) , %s, %s, %s);",
                        [country_code,year_code,electricity_urban,electricity_rural,electricity_total])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

populate_electricity_access(df)

connected


In [25]:
def populate_energy_output(df):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()

        print('connected')
        for i in range(len(df)):

            row_dict = df.loc[i]
            country_code = row_dict['Country Code']
            year_code = row_dict['Time Code']
            renewable_output_gwh = row_dict['Renewable electricity output (GWh) [4.1.2_REN.ELECTRICITY.OUTPUT]']
            total_output_gwh = row_dict['Total electricity output (GWh) [4.1.1_TOTAL.ELECTRICITY.OUTPUT]']

            if renewable_output_gwh or total_output_gwh :
                cur.execute(
                    "INSERT INTO energy_output \
                        (country_year_id, renewable_output_gwh, total_output_gwh) \
                    VALUES ( \
                        (SELECT country_year.id FROM country_year WHERE country_code = %s AND year_code = %s) , %s, %s);",
                        [country_code,year_code,renewable_output_gwh,total_output_gwh])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

populate_energy_output(df)

connected


In [26]:
def populate_energy_consumption(df):
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()

        print('connected')
        for i in range(len(df)):

            row_dict = df.loc[i]
            country_code = row_dict['Country Code']
            year_code = row_dict['Time Code']
            renewable_consumption_tj = row_dict['Renewable energy consumption (TJ) [3.1_RE.CONSUMPTION]']
            total_consumption_tj = row_dict['Total final energy consumption (TFEC) (TJ) [1.1_TOTAL.FINAL.ENERGY.CONSUM]']

            if renewable_consumption_tj or total_consumption_tj :
                cur.execute(
                    "INSERT INTO energy_consumption \
                        (country_year_id, renewable_consumption_tj, total_consumption_tj) \
                    VALUES ( \
                        (SELECT country_year.id FROM country_year WHERE country_code = %s AND year_code = %s) , %s, %s);",
                        [country_code,year_code,renewable_consumption_tj,total_consumption_tj])
        conn.commit() 
        cur.close() 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

populate_energy_consumption(df)

connected
